In [1]:
import json
import pandas as pd
from collections import Counter
from tqdm import tqdm
from github import Github
from get_latest_issues import get_closer_commits

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
df = pd.read_csv("prediction_real_world_issues.csv")
print(len(df))
df.head()

NameError: name 'pd' is not defined

In [ ]:
Counter(df.status_code)

Counter({'not-task': 85,
         'gfi': 67,
         'in-progress': 115,
         'in-discussion': 70,
         'reported': 27,
         'closed': 9,
         'reported,confirmed': 16,
         'not-gfi': 28,
         'reported,denied': 28,
         'not task': 25,
         'skipped': 41})

In [ ]:
commits = []
closed = []
config = json.load(open("config.json", "r"))
gh = Github(config["tokens"][0])
for url in tqdm(df.url):
    repo_name = url.split("/")[3] + "/" + url.split("/")[4]
    num = int(url.split("/")[-1])
    repo = gh.get_repo(repo_name)
    issue = repo.get_issue(num)
    if issue.state != "closed":
        commits.append(-1)
        closed.append(False)
        continue
    closed.append(True)
    closer_commits = get_closer_commits(gh, issue)
    commits.append(-1 if closer_commits is None else closer_commits)
df["closed"] = closed
df["commits"] = commits                           

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 511/511 [07:41<00:00,  1.11it/s]


In [ ]:
df.head(100)

,scores,url,status_note,status_code,closed,commits
0,0.828901,https://github.com/moment/moment/issues/5923,not-task,not-task,False,-1
1,0.838787,https://github.com/moment/moment/issues/5925,not-task,not-task,False,-1
2,0.753330,https://github.com/mui-org/material-ui/issues/...,gfi,gfi,False,-1
3,0.859697,https://github.com/mui-org/material-ui/issues/...,in-progress,in-progress,False,-1
4,0.820086,https://github.com/mui-org/material-ui/issues/...,in-progress,in-progress,False,-1
...,...,...,...,...,...,...
95,0.805930,https://github.com/facebook/docusaurus/issues/...,"reported,denied","reported,denied",False,-1
96,0.812172,https://github.com/facebook/docusaurus/issues/...,not-task,not-task,False,-1
97,0.809053,https://github.com/facebook/docusaurus/issues/...,not-gfi,not-gfi,False,-1
98,0.816516,https://github.com/facebook/docusaurus/issues/...,not-task,not-task,True,-1


In [ ]:
Counter(df.closed)

Counter({False: 446, True: 65})

In [ ]:
Counter(df.commits)

Counter({-1: 486,
         1: 10,
         20: 1,
         4: 1,
         3: 1,
         33: 2,
         35: 1,
         0: 1,
         1703: 1,
         493: 1,
         14: 1,
         309: 1,
         5: 1,
         2: 1,
         283: 1,
         161: 1})

In [ ]:
Counter(df.commits[df.status_code == "gfi"])

Counter({-1: 61, 1: 2, 20: 1, 309: 1, 5: 1, 2: 1})

In [ ]:
Counter(df.commits[df.status_code == "reported"])

Counter({-1: 26, 493: 1})

In [ ]:
Counter(df.commits[df.status_code == "reported,confirmed"])

Counter({-1: 15, 1: 1})

In [ ]:
Counter(df.commits[df.status_code == "reported,denied"])

Counter({-1: 26, 4: 1, 161: 1})

In [ ]:
df.to_csv("prediction_real_world_issues.csv", index=False)